<a href="https://colab.research.google.com/github/korakoe/VALL-E-X/blob/main/finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
# @title Mount Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Download Dependancies

In [1]:
# @title Clone Repo

!git clone https://github.com/0417keito/VALL-E-X-Trainer-by-CustomData.git
%cd VALL-E-X-Trainer-by-CustomData

Cloning into 'VALL-E-X-Trainer-by-CustomData'...
remote: Enumerating objects: 1072, done.
remote: Counting objects: 100% (262/262), done.
remote: Compressing objects: 100% (89/89), done.
remote: Total 1072 (delta 188), reused 173 (delta 173), pack-reused 810
Receiving objects: 100% (1072/1072), 58.06 MiB | 26.21 MiB/s, done.
Resolving deltas: 100% (325/325), done.
/content/VALL-E-X-Trainer-by-CustomData


In [4]:
# @title Install Requirements
!pip install -r requirements.txt
!pip install torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 13.6 MB/s eta 0:00:00


# Create Dataset
Skip this section if you have already processed a dataset
<br>
<br>
Dataset should be formatted like this, creating the dataset will automatically transcribe it
```
data_dir
├── bpe_69.json
├── utt1.wav
├── utt2.wav
├── utt3.wav
......
└── utt{n}.wav
```




In [2]:
# @title Create Dataset (SLOW! - ONLY NEEDS TO BE DONE ONCE - LOAD FROM DRIVE IF POSSIBLE)
data_path = "/content/drive/MyDrive/test" # @param {type:"string"}

%cd VALL-E-X-Trainer-by-CustomData/

import wget
from customs.make_custom_dataset import create_dataset

'''
How should the data_dir be created?
Place the necessary audio files in data_dir.
Transcription, tokenization, etc. of the audio files are done by the create_dataset function.

data_dir
├── bpe_69.json
├── utt1.wav
├── utt2.wav
├── utt3.wav
......
└── utt{n}.wav
'''

wget.download("https://raw.githubusercontent.com/0417keito/VALL-E-X-Trainer-by-CustomData/master/utils/g2p/bpe_69.json", data_path)

create_dataset(data_path, dataloader_process_only=True)

/content/VALL-E-X-Trainer-by-CustomData


/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
Downloading: "https://dl.fbaipublicfiles.com/encodec/v0/encodec_24khz-d7cc33bc.th" to /root/.cache/torch/hub/checkpoints/encodec_24khz-d7cc33bc.th
100%|██████████| 88.9M/88.9M [00:00<00:00, 94.7MB/s]


# Begin Training

In [7]:
# @title Train Model
data_path = "/content/drive/MyDrive/test" # @param {type:"string"}
validation_path = "/content/drive/MyDrive/test" # @param {type:"string"}
dtype = "bfloat16" # @param ["float32", "float16", "bfloat16"]
exp_dir = "/content/drive/MyDrive/valle/out" # @param {type:"string"}
model_name = "valle" # @param {type:"string"}
max_duration = 40 # @param {type:"integer"}
epochs = 30 # @param {type:"integer"}
start_from_pretrained_checkpoint = 0 # @param {type:"integer"}
base_lr = 1e-4 # @param {type:"number"}
save_every_n_steps = 10000 # @param {type:"integer"}
valid_every_n_steps = 10000 # @param {type:"integer"}
keep_last = 3 # @param {type:"integer"}
grad_accum = 1 # @param {type:"integer"}


!python train.py \
--train_dir $data_path \
--valid_dir $validation_path \
--max-duration $max_duration \
--model-name $model_name \
--exp-dir $exp_dir \
--dtype $dtype \
--num_epochs $epochs \
--start-epoch $start_from_pretrained_checkpoint \
--base-lr $base_lr \
--save-every-n $save_every_n_steps \
--valid_interval $valid_every_n_steps \
--keep-last-k $keep_last \
--accumulate-grad-steps $grad_accum \
--visualise True \


2023-11-04 04:40:02.304580: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-04 04:40:02.304644: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-04 04:40:02.304693: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-04 04:40:04.636134: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprec